License
---
This file is part of CPAchecker,<br>
a tool for configurable software verification:<br>
https://cpachecker.sosy-lab.org<br>
<br>
SPDX-FileCopyrightText: 2020 Dirk Beyer <https://www.sosy-lab.org><br>
<br>
SPDX-License-Identifier: Apache-2.0<br>
<br>
---


Instructions
---
*description*
<br>You can set a threshold to substract the start-up time of CPAchecker.

In [ ]:
# threshold in seconds. 
# Increase the time limit by <threshold> seconds to substract the time CPAchecker needs to start. Only applied on sequential analysis.
threshold = 5

This cell provides the util methods to convert the result sets to pandas dataframe format.

In [312]:
from benchexec import tablegenerator
import benchexec.result
import pandas as pd
import numpy as np
from typing import Optional, List, Union, Tuple
from dataclasses import dataclass
import os

def get_data(data_files: Union[List[str], str], disable_fv=True) -> List[pd.DataFrame]:
    if not isinstance(data_files, str):
        return [d for f in data_files for d in get_data(f)]
    if not data_files.endswith(".xml.bz2"):
        data_files = data_files if data_files.startswith("/") else "/" + data_files
        path = os.getcwd() + data_files
        if not os.path.isdir(path):
            print("Argument is not a directory or a xml.bz2 file:", data_files)
            return []
        files = os.listdir(path)
        return get_data([os.path.join(data_files, file)[1:] for file in files if file.endswith(".xml.bz2")])
    raw_data = _load_data(data_files) if disable_fv else [res for res in _load_data_with_fv(data_files) if res is not None][0]
    return [pd.DataFrame(data=_get_values(raw_data), columns=_get_column_titles(raw_data))]


def _load_data(data_file: str) -> tablegenerator.RunSetResult:
    parser = tablegenerator.create_argument_parser()
    options = parser.parse_args([data_file])
    return tablegenerator.load_result(data_file, options)

def _load_data_with_fv(data_file: str, table='table.xml'):
    parser = tablegenerator.create_argument_parser()
    options = parser.parse_args([data_file, '-x', table])
    table_definition = tablegenerator.parse_table_definition_file(table)
    return tablegenerator.load_results_with_table_definition([data_file], table_definition, table, options)

def _get_column_titles(result_set: tablegenerator.RunSetResult):
    return ['task_id', 'category'] + [col.title for col in result_set.columns]


def _get_values(result_set):
    return [[r.task_id[0], r.category] + cast_values(r) for r in result_set.results]


def cast_values(r: tablegenerator.RunResult):
    return [to_value(value, column) for value, column in zip(r.values, r.columns)]


def to_value(value, column):
    if not column.is_numeric() or value is None:
        return value
    if column.unit is None:
        return value
    return float(value[:-len(column.unit)])

def merge_columns(category, status):
    merge_dict = {
        ("correct", "true"): "TP",
        ("incorrect", "true"): "FP",
        ("correct", "false"): "TN",
        ("incorrect", "true"): "FN"
    }
    if (category, status) in merge_dict:
        return merge_dict[(category, status)]
    return "ERR/UNKNOWN/TIMEOUT"

def read_benchmarks(data_files: Union[List[str], str]):
    frames = []
    for df in get_data(data_files):
        # skip tables with missing columns (e.g. analysis was aborted)
        if not {'cputime', 'status', 'category'}.issubset(df.columns):
            print("Skipped table because it is missing columns. The first 5 rows: ")
            print(df.head(5))
            frames.append(pd.DataFrame(columns=['task_id', 'result', 'cputime']))
            continue
        # skip aborted tasks
        df = df[df['category'] != "aborted"]
        df = df.drop(columns=['host'], errors='ignore')
        df['result'] = df.apply(lambda row: merge_columns(row['category'], row['status']), axis=1)
        frames.append(df[['task_id', 'result', 'cputime']])
    return frames

The following methods are used to work with feature vectors.

In [ ]:
@dataclass
class FeatureVector:
    has_floats: Optional[bool] = None
    has_array: Optional[bool] = None
    has_recursion: Optional[bool] = None
    has_composite_handling: Optional[bool] = None
    has_loop: Optional[bool] = None
    has_alias_handling: Optional[bool] = None
        
    default = FeatureVector.from_tuple((-1,-1,-1,-1,-1,-1))
        
    def to_tuple(self):
        lookup = {
            None : -1,
            True : 1,
            False: 0
        }
        return (lookup[self.has_floats],
                lookup[self.has_array],
                lookup[self.has_recursion],
                lookup[self.has_composite_handling],
                lookup[self.has_loop],
                lookup[self.has_alias_handling])
    
    def matches(self, fv):
        t1 = self.to_tuple()
        t2 = fv.to_tuple()
        for elem in zip(t1,t2):
            if elem[0] == -1 or elem[1] == -1:
                continue
            if (elem[0] != elem[1]):
                return False
        return True
    
    def __eq__(self, other):
        if isinstance(other, FeatureVector):
            return self.matches(other)
        return False
    
    def __hash__(self):
        return hash(self.to_tuple())
    
    def find_result_set(self, lookup):
        for key in lookup:
            compare = key
            if isinstance(key, tuple):
                compare = FeatureVector.from_tuple(key)
            if (compare == self):
                return lookup[key]
        return None
    
    @staticmethod
    def from_tuple(fv_tuple):
        lookup = {
            -1: None,
             0: False,
             1: True
        }
        param_tuple = (lookup.get(t, None) for t in fv_tuple)
        return FeatureVector(*param_tuple)
    
    @staticmethod
    def from_dataframe_row(row):
        return FeatureVector.from_tuple((row['has_floats'],
                                         row['has_arrays'],
                                         row['has_recursion'],
                                         row['has_composite_handling'],
                                         row['has_loop'],
                                         row['has_alias_handling']))

def config_to_df(lookup):
    config = dict()
    for key in lookup:
        config[key] = get_data(lookup[key])[0]
    return config

def find_fitting_results(data_files: Union[List[str], str], lookup):
    # TODO create fvs
    config = config_to_df(lookup)
    fv_tables = get_data(data_files, disable_fv=False)
    frames = []
    for fv_table in fv_tables:
        df = pd.DataFrame(columns=fv_table.columns)
        for index, row in fv_table.iterrows():
            task_id = row["task_id"]
            fv = FeatureVector.from_dataframe_row(row)
            table = fv.find_result_set(config)
            table = table[table["task_id"]==task_id]
            df = pd.concat([df,table])
        frames.append(df)
        break
    return frames

The following methods are used to calculate the final score.

In [ ]:
def prepare_frames(input_sets, sequential=False):
    frames = []
    for (df, time) in input_sets:
        epsilon = 0 if not frames or not sequential else threshold
        df['result'] = np.where((df['cputime'] > time) & (time != -1), 'ERR/UNKNOWN/TIMEOUT', df['result'])
        df['cputime'] = np.where((df['cputime'] > time) & (time != -1), time, df['cputime'])
        frames.append(df)
    concat = pd.concat(frames)
    return concat[['task_id', 'result', 'cputime']]

def sequential(input_sets):
    prepared = prepare_frames(input_sets, sequential=True)
    distinct_ids = prepared.task_id.unique()
    results = pd.DataFrame(columns = prepared.columns)
    # loop through every distinct key
    for key in distinct_ids:
        selection = prepared[prepared['task_id'] == key]
        col_result_distinct = selection.result.unique()
        # if all results are unknown the tasks takes sum(cputime)
        if (len(col_result_distinct) == 1):
            if (col_result_distinct[0] == "ERR/UNKNOWN/TIMEOUT"):
                sum_time = selection['cputime'].sum()
                new_row = pd.DataFrame([[key, "ERR/UNKNOWN/TIMEOUT", sum_time]], columns=prepared.columns)
                results = results.append(new_row)
                continue
        # otherwise return result of first analysis that finishes (sum cputime until result != "ERR/UNKNOWN/TIMEOUT")
        col_result = ""
        sum_time = 0
        for index, row in selection.iterrows():
            if (row["result"] != "ERR/UNKNOWN/TIMEOUT"):
                col_result = row["result"]
                sum_time = sum_time + row["cputime"]
                break
            sum_time = sum_time + row["cputime"]
        new_row = pd.DataFrame([[key, col_result, sum_time]], columns=prepared.columns)
        results = results.append(new_row)
    return results

def parallel(input_sets, timelimit=-1):
    prepared = prepare_frames([(df, timelimit) for df in input_sets])
    prepared = prepared[['task_id', 'result', 'cputime']]
    distinct_ids = prepared.task_id.unique()
    results = pd.DataFrame(columns = prepared.columns)
    # loop through every distinct key
    for key in distinct_ids:
        selection = prepared[prepared['task_id'] == key]
        col_result_distinct = selection.result.unique()
        # if all results are unknown the tasks takes max(cputime)
        if (len(col_result_distinct) == 1):
            if (col_result_distinct[0] == "ERR/UNKNOWN/TIMEOUT"):
                max_time = selection['cputime'].max()
                new_row = pd.DataFrame([[key, "ERR/UNKNOWN/TIMEOUT", max_time]], columns=prepared.columns)
                results = results.append(new_row)
                continue
        # otherwise return result of first analysis that finishes min(cputime | result != "UNKNOWN")
        selection = selection[selection['result'] != "ERR/UNKNOWN/TIMEOUT"]
        selection = selection[selection['cputime'] == selection['cputime'].min()].head(1)
        results = results.append(selection)
    return results

def evaluate(df):
    counts = df.result.value_counts()
    average_time = int(df.cputime.mean())
    time = pd.Series([average_time], index=['Average time'])
    return counts.append(time)

There are 2 strategies available:
- sequential:
```
simulates the sequential execution of all given analysis
| analysis 0 | analysis 1 | ... | analysis n |
```
<br>
- parallel:
```file2
simulates the parallel execution of all given analysis
|      analysis 0     |
|        analysis 1        |
|       analysis 2       |
```

All analysis simulate an execution where the calculation terminates as soon as a result different from "UNKNOWN", "ERROR" or "TIMEOUT" occurs (if possible).

The format of the input should look like this:
```python
[sub1, sub2, sub3, sub4] = read_benchmarks(
            ['path/to/file1.xml.bz2', 
              'path/to/file2.xml.bz2', 
              'path/to/file3.xml.bz2', 
              'path/to/file4.xml.bz2'
             ]
)
```
**Note**: You can pass a folder located in the same directory as this file as well. Pass "folder/" to run the script with all files in the specified folder.

In [ ]:
# define your tasks here
[sub1] = read_benchmarks(
            ['results/featureExtraction.2020-11-13_14-08-04.results.SV-COMP21_unreach-call.ReachSafety-ECA.xml.bz2']
)

Create a configuration for feature vectors. A vectors has 6 attributes with values `True, False, None`. You can either create a `FeatureVector` by hand or simply use a tuple. The first entry of the tuple sets the value for `has_floats` the last entry sets the value for `has_alias_handling`. The tuples are based on integer values. `0` will be translated to `False`, `1` to `True` and every other integer to `None`. `None` means that you don't care which value is on this position of the feature vector.

The class `FeatureVector` (here you can see which entry of the tuple belongs to which attribute):
```python
@dataclass
class FeatureVector:
    has_floats: Optional[bool] = None
    has_array: Optional[bool] = None
    has_recursion: Optional[bool] = None
    has_composite_handling: Optional[bool] = None
    has_loop: Optional[bool] = None
    has_alias_handling: Optional[bool] = None
```

Here you can see an example for all valid configuration. The tuples will automatically be transformed to `FeatureVectors`.
```python
configuration = {
    (1,0,0): "path/to/result1",
    (1,0,1,0,1,0) : "path/to/result1",
    FeatureVector.from_tuple((1,1,1,1,0,1)): "path/to/result3",
    FeatureVector(has_floats=True, has_loop=False): "path/to/result4",
    FeatureVector.default: "path/to/default/result"
}
```

In [ ]:
# define your configuration here
configuration = {
    FeatureVector.default: "results/featureExtraction.2020-11-13_14-08-04.results.SV-COMP21_unreach-call.ReachSafety-ProductLines.xml.bz2"
}

In [ ]:
# create a dataframe based on feature vectors:
[subN] = find_fitting_results('results/featureExtraction.2020-11-13_14-08-04.results.SV-COMP21_unreach-call.ReachSafety-Recursive.xml.bz2', configuration)

Now all your tables are dataframes and you can use the provided methods. Compose your tasks, for example, as follows:
```python
strategy = sequential(
        [
           (parallel([sub1, sub2], timelimit0=200), timelimit1),
           (subN, timelimit2)
        ]
)         
```
Set `timelimitN = -1` to use the actual time limit of the analysis.

In [ ]:
#strategy = sequential(result_sets)
strategy = sequential(dirname = os.path.dirname(__file__)
    [
        (parallel([sub1, sub3]), 400),
        (parallel([sub2, sub4]), 800)
    ]
)

Now you can rate your chosen timelimits. The output shows how often a result (e.g. "UNKNOWN, "TP", "FN", ...) occured. "Average time" displays the mean of the cputime in seconds to find a result.

In [ ]:
evaluate(strategy)